In [87]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import requests
import re
import pandas as pd
import numpy as np
import os

In [88]:
def cantidad_fechas(driver):

    elem = driver.find_element(By.ID, "fixtseccion")
    cant_fechas = len(elem.find_elements(By.XPATH, '//div[@class="cfecha"]'))+1
    
    return driver, cant_fechas

In [89]:
def pretify_text(text):
    return text.replace("         ", "").replace("        ", "").replace("       ", "").replace("\n", "").replace("%", "")

In [90]:
def tarjetas(soup):
    regex = re.compile('.*images/amarilla.png.*')
    amarillas_local = soup.find("table", {"id": ["formacion1"]}).findAll("img", {"src": [regex]})
    if len(amarillas_local) > 0:
        amarillas_local = len(amarillas_local) - 1
    else:
        amarillas_local = 0
    amarillas_visitante = soup.find("table", {"id": ["formacion2"]}).findAll("img", {"src": [regex]})
    if len(amarillas_visitante) > 0:
        amarillas_visitante = len(amarillas_visitante) - 1
    else:
        amarillas_visitante = 0

    regex = re.compile('.*images/roja.png.*')
    rojas_local = soup.find("table", {"id": ["formacion1"]}).findAll("img", {"src": [regex]})
    if len(rojas_local) > 0:
        rojas_local = len(rojas_local) - 1
    else:
        rojas_local = 0
    rojas_visitante = soup.find("table", {"id": ["formacion2"]}).findAll("img", {"src": [regex]})
    if len(rojas_visitante) > 0:
        rojas_visitante = len(rojas_visitante) - 1
    else:
        rojas_visitante = 0
    
    return amarillas_local, amarillas_visitante, rojas_local, rojas_visitante

In [91]:
def info(soup):
    data_local = soup.findAll("div", {"id": ["porcentaje1"]})
    if len(data_local) != 0:
        posesion_local = pretify_text(data_local[0].get_text())
        tiros_arco_local = pretify_text(data_local[1].get_text())
        intentos_local = pretify_text(data_local[2].get_text())
        fouls_local = pretify_text(data_local[3].get_text())
        corners_local = pretify_text(data_local[4].get_text())
    else:
        posesion_local = None
        tiros_arco_local = None
        intentos_local = None
        fouls_local = None
        corners_local = None

    data_visitante = soup.findAll("div", {"id": ["porcentaje2"]})
    if len(data_visitante) != 0:
        posesion_visitante = pretify_text(data_visitante[0].get_text())
        tiros_arco_visitante = pretify_text(data_visitante[1].get_text())
        intentos_visitante = pretify_text(data_visitante[2].get_text())
        fouls_visitante = pretify_text(data_visitante[3].get_text())
        corners_visitante = pretify_text(data_visitante[4].get_text())
    else:
        posesion_visitante = None
        tiros_arco_visitante = None
        intentos_visitante = None
        fouls_visitante = None
        corners_visitante = None
    return posesion_local, tiros_arco_local, intentos_local, fouls_local, corners_local, posesion_visitante, tiros_arco_visitante, intentos_visitante, fouls_visitante, corners_visitante

In [92]:
def equipos(soup):
    equipos = soup.findAll("td", {"class": ["nomequipo"]})
    equipo_local = pretify_text(equipos[0].get_text())
    equipo_visitante = pretify_text(equipos[1].get_text())

    return equipo_local, equipo_visitante

In [93]:
def goles(soup):
    goles_local = soup.find("div", {"id": ["ficha-resultado1"]}).get_text().replace(" ", "").replace("\n", "")
    goles_visitante = soup.find("div", {"id": ["ficha-resultado2"]}).get_text().replace(" ", "").replace("\n", "")

    return goles_local, goles_visitante

In [94]:
def obtener_partidos(driver):
    
    elem = driver.find_element(By.ID, "fixtseccion")
    filter = f'//div[@class="cfecha"][text()="{i}"]'
    link = elem.find_element(By.XPATH, filter)
    link.click()
    time.sleep(1.5)

    elems = driver.find_element(By.ID, "fixturein")
    elems = elems.find_elements(By.CLASS_NAME, "game-info")[1:]
    links = []
    for elem in elems:
        href = elem.get_attribute("innerHTML")
        if elem.get_attribute("innerHTML") != "":
            href = elem.find_element(By.XPATH, ".//a[@href]").get_attribute("href")
            links.append(href)
        else:
            links.append(elem.get_attribute("id")[3:])
    
    return driver, links

In [95]:
def partido_info_desde_torneo(driver, game):
    equipo_local = driver.find_element(By.XPATH, f"//span[@id='t1{game}']").text
    equipo_visitante = driver.find_element(By.XPATH, f"//span[@id='t2{game}']").text

    goles_local = driver.find_element(By.XPATH, f"//span[@id='r1{game}']").text
    goles_visitante = driver.find_element(By.XPATH, f"//span[@id='r2{game}']").text

    rojas_local = driver.find_elements(By.XPATH, f"//div[@id='roj1{game}']")
    rojas_local = len(rojas_local)-1
    rojas_visitante = driver.find_elements(By.XPATH, f"//div[@id='roj2{game}']")
    rojas_visitante = len(rojas_visitante)-1

    return equipo_local, equipo_visitante, goles_local, goles_visitante, rojas_local, rojas_visitante

In [97]:
promiedos = []

torneos = [
    "https://www.promiedos.com.ar/torneo=1042",
    "https://www.promiedos.com.ar/torneo=1041",
    "https://www.promiedos.com.ar/torneo=1025",
    "https://www.promiedos.com.ar/torneo=1024",
    "https://www.promiedos.com.ar/torneo=1023",
    "https://www.promiedos.com.ar/torneo=1022",
    "https://www.promiedos.com.ar/torneo=1021",
    "https://www.promiedos.com.ar/torneo=1020",
    "https://www.promiedos.com.ar/torneo=1018",
    "https://www.promiedos.com.ar/torneo=1014",
    "https://www.promiedos.com.ar/torneo=1013",
    "https://www.promiedos.com.ar/torneo=1012",
    "https://www.promiedos.com.ar/torneo=1011",
    "https://www.promiedos.com.ar/torneo=1010",
    "https://www.promiedos.com.ar/torneo=1009",
    "https://www.promiedos.com.ar/torneo=1008",
    "https://www.promiedos.com.ar/torneo=1007",
    "https://www.promiedos.com.ar/torneo=1097",
]

for torneo in torneos:
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(torneo)

    nombre_torneo = driver.find_element(By.ID, "titulo1").text
    print("Nombre del torneo:", nombre_torneo)

    driver, cant_fechas = cantidad_fechas(driver)
    print("-Cantidad de fechas a escrapear:", cant_fechas)

    for i in range(1, cant_fechas+1):
        driver, partidos = obtener_partidos(driver)
        if i == 1:
            print("--Cantidad de partidos por fecha:", len(partidos))
        print("-Scrapeando fecha", i, " de", cant_fechas, "de torneo", nombre_torneo)

        j = 1
        for partido in partidos:
            print("--Scrapeando partido", j, " de", len(partidos), end="\r", flush=True)
            
            if "promiedos" in partido:
                page = requests.get(partido, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36'})
                soup = BeautifulSoup(page.content, "html.parser")
                soup = BeautifulSoup(soup.prettify(), "html.parser")

                equipo_local, equipo_visitante = equipos(soup)

                goles_local, goles_visitante = goles(soup)

                posesion_local, tiros_arco_local, intentos_local, fouls_local, corners_local, posesion_visitante, tiros_arco_visitante, intentos_visitante, fouls_visitante, corners_visitante = info(soup)

                amarillas_local, amarillas_visitante, rojas_local, rojas_visitante = tarjetas(soup)


            else:
                equipo_local, equipo_visitante, goles_local, goles_visitante, rojas_local, rojas_visitante = partido_info_desde_torneo(driver, partido)

                posesion_local, posesion_visitante = np.nan, np.nan
                tiros_arco_local, tiros_arco_visitante = np.nan, np.nan
                intentos_local, intentos_visitante = np.nan, np.nan
                fouls_local, fouls_visitante = np.nan, np.nan
                corners_local, corners_visitante = np.nan, np.nan
                amarillas_local, amarillas_visitante = np.nan, np.nan

            promiedos.append({
                "torneo": nombre_torneo,
                "fecha": i,
                "partido": j,
                "equipo_local": equipo_local,
                "equipo_visitante": equipo_visitante,
                "goles_local": goles_local,
                "goles_visitante": goles_visitante,
                "posesion_local": posesion_local, 
                "tiros_arco_local": tiros_arco_local, 
                "intentos_local": intentos_local, 
                "fouls_local": fouls_local, 
                "corners_local": corners_local, 
                "posesion_visitante": posesion_visitante, 
                "tiros_arco_visitante": tiros_arco_visitante, 
                "intentos_visitante": intentos_visitante, 
                "fouls_visitante": fouls_visitante, 
                "corners_visitante": corners_visitante,
                "amarillas_local": amarillas_local, 
                "amarillas_visitante": amarillas_visitante, 
                "rojas_local": rojas_local, 
                "rojas_visitante": rojas_visitante
            })
            j += 1
        print("\n")
    driver.close()

df = pd.DataFrame(promiedos)

for torneo in df.torneo.unique():
    if df.loc[df.torneo == torneo].rojas_local.sum() == 0 and df.loc[df.torneo == torneo].rojas_visitante.sum() == 0:
        df.loc[df.torneo == torneo, "rojas_local"] = np.nan
        df.loc[df.torneo == torneo, "rojas_visitante"] = np.nan

df.to_csv("promiedos_all_data.csv", index = False)